In [13]:
from rdflib import Graph, Dataset, URIRef
import re
import time

In [4]:
# the URI for our named graph that will contain our data
# CHANGE URI please to make it unique
anno_named_graph_uri = 'http://example.org/ns/users/robcast/genesis-example'
anno_named_graph_uriref = URIRef(anno_named_graph_uri)

In [5]:
# read the JSON-LD file with annotations from Recogito into a Graph
anno_graph = Graph()
anno_graph.parse('recogito-genesis-annotations.jsonld', 
                format='json-ld',
                publicId=anno_named_graph_uriref)

<Graph identifier=Nc5ed7fe78ce74982bc70d1df1e174142 (<class 'rdflib.graph.Graph'>)>

In [6]:
# write the Graph with annotations to a n-triples file
anno_graph.serialize('recogito-genesis-annotations.nt', format='nt', encoding='utf-8')

<Graph identifier=Nc5ed7fe78ce74982bc70d1df1e174142 (<class 'rdflib.graph.Graph'>)>

In [8]:
# read the n-triples file and replace all blank nodes (starting with _:) with real URIs
# and write that to a new file
with open('recogito-genesis-annotations.nt', 'r') as infile, open('recogito-genesis-annotations-nb.nt', 'w') as outfile:
    for line in infile:
        line = re.sub(r'_:([^ ]+)', '<' + anno_named_graph_uri + r'/bnode_\1>', line)
        print(line, file=outfile)

In [15]:
# open a connection to the Blazegraph triple store
blazegraph_uri = 'http://localhost:8080/blazegraph/namespace/kb/sparql'
store = Dataset('SPARQLUpdateStore')
store.open((blazegraph_uri, blazegraph_uri))

In [16]:
# find our named graph if it exists or create a new one
anno_named_graph = None
for g in store.graphs():
    if g.identifier == anno_named_graph_uriref:
        anno_named_graph = g

if anno_named_graph is None:
    anno_named_graph = store.graph(anno_named_graph_uriref)

In [17]:
# record time
start_time = time.time()

# parse our modified n-triples file into our named graph (takes a while)
anno_named_graph.parse('recogito-genesis-annotations-nb.nt')

print(f"loading triples done in {time.time() - start_time}s")

loading triples done in 277.6832768917084s


In [18]:
store.close()